In [ ]:
# Block 0: Documentation

print('Code to open, read the metadata, and understand the contents of an ABI L2 AOD data file\n')
print('Includes a function to calculate latitude and longitude in degrees from the GOES ABI fixed grid projection variables and contstants in an ABI file\n')
print('Version 2.0, November 16, 2021\n')
print('Written using using Python v3.9 and netCDF4 v1.5.7\n')
print('Author: Dr. Amy Huff (IMSG at NOAA/NESDIS/STAR), amy.huff@noaa.gov\n')
print('Block 1 imports modules and libraries, and Block 2 opens the data file; there is no visible output from these blocks. Blocks 2-10 are stand-alone commands to open/view metadata and data in the file. Blocks 11-12 demonstrate how to calculate latitude and longitude from the GOES ABI fixed grid projection data.\n')
print('**Please acknowledge the NOAA/NESDIS/STAR Aerosols and Atmospheric Composition Science Team if using any of this code in your work/research!**')

In [ ]:
# Block 1: Import Python packages

# Library to work with netCDF files
from netCDF4 import Dataset

# Library to perform array operations
import numpy as np 

# Module to set filesystem paths appropriate for user's operating system
from pathlib import Path

In [ ]:
# Block 2: Open an ABI aerosol optical depth (AOD) netCDF4 data file
# Ensure the data file is in the current working directory(cwd)!

# Set file name and directory where file is located
# We use the pathlib.Path module to set directory to avoid potential errors arising from user's OS
file_path = Path.cwd()  # Current working directory
file_name = 'OR_ABI-L2-AODF-M6_G16_s20222501400205_e20222501409513_c20222501412051.nc'  # One of the files we downloaded
fname = file_path / file_name  # This is the full file name (directory + file) using pathlib.Path syntax

# Open the specified file using the netCDF4 library
file_id = Dataset(fname)

In [ ]:
# Block 3: Print the metadata for the entire file

print(file_id)

In [ ]:
# Block 4: Print the "AOD" variable metadata

print(file_id.variables['AOD'])

In [ ]:
# Block 5: Convert the AOD valid range given in metadata from unsigned integers to floats
# Metadata indicate AOD data are stored as unsigned integers ("uint16")

print('AOD range is', 
      (file_id.variables['AOD'].valid_range)*(file_id.variables['AOD'].scale_factor)+(file_id.variables['AOD'].add_offset))

In [ ]:
# Block 6: Print the AOD data array and check the range of values
# Note: AOD data are unitless ("units" = 1)

# Read in AOD data
# Metadata indicates data are 2-dimensional so use [:,:] to extract all data along axis-0, axis-1
# Note: netCDF4 library automatically masks pixels (--) with missing data (i.e., "_FillValue") and applies the
# "scale_factor" and "add_offset" to convert stored integers to floats
abi_aod = file_id.variables['AOD'][:,:]

# Print an excerpt of the AOD array (near the center) to see values
print(abi_aod[2710:2714, 2710:2714])

# Skip a line
print('\n')

# Print max and min of AOD array to check data range
print('The maximum AOD value is', np.max(abi_aod))
print('The minimum AOD value is', np.min(abi_aod))

# Check shape of array; should match dimensions of ('y', 'x')
print('The AOD array shape is', abi_aod.shape)

In [ ]:
# Block 7: Print the "DQF" variable metadata

print(file_id.variables['DQF'])

In [ ]:
# Block 8: Print the data quality flags array and check the range of values

# Read in data quality flags
# Metadata indicates data are 2-dimensional so use [:,:] to extract all data along axis-0, axis-1
# Note: netCDF4 library automatically masks pixels (--) with missing data (i.e., "_FillValue")
abi_dqf = file_id.variables['DQF'][:,:]

# an excerpt of the DQF array (near the center) to see values
print(abi_dqf[2710:2714, 2710:2714])

# Skip a line
print('\n')

# Print max and min of DQF array to check data range
print('The maximum DQF value is', np.max(abi_dqf))
print('The minimum DQF value is', np.min(abi_dqf))

# Check shape of array; should match dimensions of ('y', 'x')
print('The DQF array shape is', abi_dqf.shape)

In [ ]:
# Block 9: Print metadata and array values for GOES ABI fixed grid projection x-coordinate variable

# Print metadata
print(file_id.variables['x'])

# Read in GOES ABI fixed grid projection x-coordinate data
# Metadata indicates data are 1-dimensional so use [:] to extract all data along axis-0
x_coordinate = file_id.variables['x'][:]

# Metadata indicates data are stored as integers ("int16") in the ABI file
# Note: netCDF4 library automatically applies "scale_factor" and "add_offset" to covnvert stored integers to floats
# Check data type of first value in array to confirm
print('\n')  # Skip a line
print('Array data values are type', type(x_coordinate[0]))

# Print the data array
print(x_coordinate)

# Check shape of array; should match dimensions of ('x')
print('The x-coordinate array shape is', x_coordinate.shape)

In [ ]:
# Block 10: Print metadata and array values for GOES ABI fixed grid projection y-coordinate variable

# Print metadata
print(file_id.variables['y'])

# Read in GOES ABI fixed grid projection y-coordinate data
# Metadata indicates data are 1-dimensional so use [:] to extract all data along axis-0
y_coordinate = file_id.variables['y'][:]

# Metadata indicates data are stored as integers ("int16") in the ABI file
# Note: netCDF4 library automatically applies "scale_factor" and "add_offset" to covnvert stored integers to floats
# Check data type of first value in array to confirm
print('\n')  # Skip a line
print('Array data values are type', type(y_coordinate[0]))

# Print the data array
print(y_coordinate)

# Check shape of array; should match dimensions of ('y')
print('The y-coordinate array shape is', y_coordinate.shape)

In [ ]:
# Block 11: Algorithm to calculate latitude and longitude from GOES ABI fixed grid projection data
# GOES ABI fixed grid projection is a map projection relative to the GOES satellite
# Units: latitude in °N (°S < 0), longitude in °E (°W < 0)
# See GOES-R Product User Guide (PUG) Volume 5 (L2 products) Section 4.2.8 for details + example of calculations

def degrees(file_id):
    
    # Read in GOES ABI fixed grid projection variables and constants
    x_coordinate_1d = file_id.variables['x'][:]
    y_coordinate_1d = file_id.variables['y'][:]
    projection_info = file_id.variables['goes_imager_projection']
    lon_origin = projection_info.longitude_of_projection_origin
    H = projection_info.perspective_point_height+projection_info.semi_major_axis
    r_eq = projection_info.semi_major_axis
    r_pol = projection_info.semi_minor_axis
    
    # Create 2D coordinate matrices from 1D coordinate vectors
    x_coordinate_2d, y_coordinate_2d = np.meshgrid(x_coordinate_1d, y_coordinate_1d)
    
    # Equations to calculate latitude and longitude
    lambda_0 = (lon_origin*np.pi)/180.0  
    a_var = np.power(np.sin(x_coordinate_2d),2.0) + (np.power(np.cos(x_coordinate_2d),2.0)*(np.power(np.cos(y_coordinate_2d),2.0)+(((r_eq*r_eq)/(r_pol*r_pol))*np.power(np.sin(y_coordinate_2d),2.0))))
    b_var = -2.0*H*np.cos(x_coordinate_2d)*np.cos(y_coordinate_2d)
    c_var = (H**2.0)-(r_eq**2.0)
    r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)
    s_x = r_s*np.cos(x_coordinate_2d)*np.cos(y_coordinate_2d)
    s_y = - r_s*np.sin(x_coordinate_2d)
    s_z = r_s*np.cos(x_coordinate_2d)*np.sin(y_coordinate_2d)
    
    abi_lat = (180.0/np.pi)*(np.arctan(((r_eq*r_eq)/(r_pol*r_pol))*((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y))))))
    abi_lon = (lambda_0 - np.arctan(s_y/(H-s_x)))*(180.0/np.pi)
    
    return abi_lat, abi_lon

In [ ]:
# Block 12: Print arrays of calculated latitude and longitude

# Ignore numpy error warnings for sqrt of negative number; occurs for GOES-16 ABI CONUS sector data
np.seterr(all='ignore')

# Call function to calculate latitude and longitude from GOES ABI fixed grid projection data
abi_lat, abi_lon = degrees(file_id)

# Pring an excerpt of the latitude array (near the center) to see values
print(abi_lat[2710:2714, 2710:2714])

# Skip a line
print('\n')

# Print max and min of latitude data to check data range
print('The maximum latitude value is', np.max(abi_lat), 'degrees')
print('The minimum latitude value is', np.min(abi_lat), 'degrees')

# Check shape of array, should match shape of ('y', 'x')
print('The latitude array shape is', abi_lat.shape)

# Skip a line
print('\n')

# Print an excerpt of the longitude array (near the center) to see values
print(abi_lon[2710:2714, 2710:2714])

# Skip a line
print('\n')

# Print max and min of longitude data to check data range
print('The maximum longitude value is', np.max(abi_lon), 'degrees')
print('The minimum longitude value is', np.min(abi_lon), 'degrees')

# Check shape of array; should match dimensions of ('y', 'x')
print('The longitude array shape is', abi_lon.shape)